In [ ]:
## Evaluación de Modelos de Machine Learning en Riesgo Crediticio
import pandas as pd

# Leer el archivo CSV
ruta_archivo = "C:\\Users\\danie\\01 Documents\\01 School\\06 Applied Maths and Computing UNAM\\07 Seventh Semester\\01 IMAC\\05 Project\\01 Development\\000 - AMEX DataSet [CSV] (20240929) - Kaggle_W.csv"
try:
    # Carga el archivo CSV
    datos = pd.read_csv(ruta_archivo)
    print("Archivo cargado exitosamente.")
    
    # Mostrar información general del dataset
    print("\nInformación general del dataset:")
    print(datos.info())
    
    # Mostrar las primeras filas del dataset
    print("\nPrimeras filas del dataset:")
    print(datos.head())
except FileNotFoundError:
    print(f"El archivo no se encontró en la ruta especificada: {ruta_archivo}")
except pd.errors.EmptyDataError:
    print("El archivo está vacío.")
except Exception as e:
    print(f"Ocurrió un error: {e}")
print("\nValores nulos por columna:")
print(datos.isnull().sum())
# Imputar valores nulos
# Columnas categóricas
datos['owns_car'] = datos['owns_car'].fillna('Unknown')

# Columnas numéricas
datos['no_of_children'] = datos['no_of_children'].fillna(datos['no_of_children'].median())
datos['no_of_days_employed'] = datos['no_of_days_employed'].fillna(datos['no_of_days_employed'].median())
datos['total_family_members'] = datos['total_family_members'].fillna(datos['total_family_members'].median())
datos['migrant_worker'] = datos['migrant_worker'].fillna(datos['migrant_worker'].mode()[0])
datos['yearly_debt_payments'] = datos['yearly_debt_payments'].fillna(datos['yearly_debt_payments'].median())
datos['credit_score'] = datos['credit_score'].fillna(datos['credit_score'].median())

# Verificar si quedan valores nulos
print("\n¿Existen valores nulos después de la imputación?")
print(datos.isnull().sum())
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Codificación de variables categóricas
encoder = LabelEncoder()
datos['gender'] = encoder.fit_transform(datos['gender'])
datos['owns_car'] = encoder.fit_transform(datos['owns_car'])
datos['owns_house'] = encoder.fit_transform(datos['owns_house'])

# One-hot encoding para occupation_type
datos = pd.get_dummies(datos, columns=['occupation_type'], drop_first=True)

# Normalización de columnas numéricas
columnas_numericas = [
    'age', 'net_yearly_income', 'no_of_days_employed', 'total_family_members',
    'yearly_debt_payments', 'credit_limit', 'credit_score'
]
scaler = StandardScaler()
datos[columnas_numericas] = scaler.fit_transform(datos[columnas_numericas])

# Ver resumen después de la limpieza
print("\nResumen del dataset limpio:")
print(datos.describe(include='all'))
# División del dataset en características y etiquetas
from sklearn.model_selection import train_test_split

# Eliminar columnas irrelevantes
X = datos.drop(['customer_id', 'name', 'credit_card_default'], axis=1)
y = datos['credit_card_default']

# División del dataset en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo inicial: Árbol de Decisión
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

modelo = DecisionTreeClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Predicciones
y_pred = modelo.predict(X_test)

# Evaluación
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

matriz_confusion = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

# Ajustar pesos de clase para el modelo
modelo_ponderado = DecisionTreeClassifier(random_state=42, class_weight={0: 1, 1: 5})
modelo_ponderado.fit(X_train, y_train)

# Predicciones con el modelo ajustado
y_pred_ponderado = modelo_ponderado.predict(X_test)

# Nueva matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred_ponderado)
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión (Modelo Ponderado)')
plt.show()

# Sobremuestreo con SMOTE para la clase minoritaria
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

# Entrenar un nuevo modelo con datos balanceados
modelo_smote = DecisionTreeClassifier(random_state=42)
modelo_smote.fit(X_res, y_res)

# Predicciones con el modelo entrenado con SMOTE
y_pred_smote = modelo_smote.predict(X_test)

# Matriz de confusión para el modelo con SMOTE
matriz_confusion_smote = confusion_matrix(y_test, y_pred_smote)
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_confusion_smote, annot=True, fmt='d', cmap='Greens')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión (Modelo SMOTE)')
plt.show()

# Evaluación general de ambos modelos
print("Reporte de Clasificación (Modelo Ponderado):")
print(classification_report(y_test, y_pred_ponderado))

print("Reporte de Clasificación (Modelo SMOTE):")
print(classification_report(y_test, y_pred_smote))

# Calcular el ROC-AUC para ambos modelos
roc_auc_ponderado = roc_auc_score(y_test, y_pred_ponderado)
roc_auc_smote = roc_auc_score(y_test, y_pred_smote)

print(f"ROC-AUC (Modelo Ponderado): {roc_auc_ponderado:.4f}")
print(f"ROC-AUC (Modelo SMOTE): {roc_auc_smote:.4f}")
import pandas as pd
import numpy as np

# Crear una función para calcular puntajes basados en reglas
def calcular_puntaje_fico(row):
    puntaje = 0

    # Regla 1: Ingreso anual
    if row['net_yearly_income'] > 100000:
        puntaje += 50  # Bajo riesgo
    elif row['net_yearly_income'] > 50000:
        puntaje += 30  # Riesgo medio
    else:
        puntaje += 10  # Alto riesgo

    # Regla 2: Uso del límite de crédito
    if row['credit_limit_used(%)'] < 30:
        puntaje += 50  # Bajo riesgo
    elif row['credit_limit_used(%)'] < 70:
        puntaje += 30  # Riesgo medio
    else:
        puntaje += 10  # Alto riesgo

    # Regla 3: Defaults previos
    if row['prev_defaults'] == 0:
        puntaje += 50  # Bajo riesgo
    elif row['prev_defaults'] == 1:
        puntaje += 20  # Riesgo medio
    else:
        puntaje += 0  # Alto riesgo

    # Regla 4: Edad
    if row['age'] > 45:
        puntaje += 30  # Bajo riesgo
    elif row['age'] > 30:
        puntaje += 20  # Riesgo medio
    else:
        puntaje += 10  # Alto riesgo

    # Regla 5: Propiedad de activos
    if row['owns_house'] == 1:
        puntaje += 20  # Propietario de vivienda
    if row['owns_car'] == 1:
        puntaje += 10  # Propietario de coche

    return puntaje

# Calcular puntajes para todos los clientes
datos['credit_score_fico'] = datos.apply(calcular_puntaje_fico, axis=1)

# Clasificar clientes según el puntaje
def clasificar_riesgo(puntaje):
    if puntaje >= 150:
        return 'Bajo Riesgo'
    elif puntaje >= 100:
        return 'Riesgo Medio'
    else:
        return 'Alto Riesgo'

datos['risk_category'] = datos['credit_score_fico'].apply(clasificar_riesgo)

# Mostrar distribución de categorías de riesgo
print(datos['risk_category'].value_counts())

# Evaluar contra la etiqueta real
from sklearn.metrics import classification_report

# Convertir las categorías a numéricas para comparación (Bajo Riesgo = 0, Medio/Alto Riesgo = 1)
datos['risk_numeric'] = datos['risk_category'].map({'Bajo Riesgo': 0, 'Riesgo Medio': 1, 'Alto Riesgo': 1})

# Generar reporte de clasificación comparado con la etiqueta real
print("Reporte de Clasificación (Modelo de Puntaje):")
print(classification_report(datos['credit_card_default'], datos['risk_numeric']))